In [5]:
# Import packages
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.colors as mcolors
import matplotlib.cm as cm
import matplotlib as mpl
import pandas as pd
import os
import sys
import argparse
import re
import seaborn as sns
from scipy import stats
# IMport cv2
import cv2
from skimage import io
from skimage import color
from skimage.segmentation import watershed
import scipy.io
import scipy.ndimage
import h5py

from scipy.ndimage import label, generate_binary_structure
from skimage.measure import regionprops
from skimage.io import imread, imshow

from skimage import measure, morphology
import math
from h5py_to_dict import h5py_to_dict
from getPrmDflt import getPrmDflt
from bwareaopen import bwareaopen
from get_con_seg import get_con_seg
from relabel import relabel


# Get the path to the parent directory
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
# # Append the parent directory to the system path
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
# Append the parent directory to the system path
parent_parent_dir = os.path.abspath(os.path.join(parent_dir, os.pardir))
if parent_parent_dir not in sys.path:
    sys.path.append(parent_parent_dir)
# Add to path parent_parent_dir/MLVcode
sys.path.append(os.path.join(parent_parent_dir, 'MLVcode'))
print("The current working directory is:")
print(os.getcwd())
print("The parent directory is:")
print(parent_dir)
print("The parent parent directory is:")
print(parent_parent_dir)
from mergeLineSegments import mergeLineSegments

The current working directory is:
/Users/aravind/Documents/BWL_github/July_PyMLV/pyMLV/demos/getLineDrawing
The parent directory is:
/Users/aravind/Documents/BWL_github/July_PyMLV/pyMLV/demos
The parent parent directory is:
/Users/aravind/Documents/BWL_github/July_PyMLV/pyMLV


# spDetect

In [6]:
def sticky(S, h, w, I, E, prm):
    # Get additional parameters
    maxIter = int(prm[0])
    nThreads = int(prm[1])
    sigs = float(prm[2])
    sigx = float(prm[3])
    sige = float(prm[4])
    sigc = float(prm[5])
    # Round off to 5 decimal places
    sigs = round(sigs, 5)
    sigx = round(sigx, 5)
    sige = round(sige, 5)
    sigc = round(sigc, 5)
    # Print the parameters
    print("maxIter: ", maxIter)
    print("nThreads: ", nThreads)
    print("sigs: ", sigs)
    print("sigx: ", sigx)
    print("sige: ", sige)
    print("sigc: ", sigc)

    # Initialize mus and ns
    m = int(np.max(S)) + 1
    ns = np.zeros(m, dtype=np.float32)
    mus = np.zeros((m, 5), dtype=np.float32)

    for x in range(w):
        for y in range(h):
            i = S[y, x]
            ns[i] += 1
            mus[i, 0] += x
            mus[i, 1] += y
            for z in range(3):
                mus[i, z + 2] += I[y, x, z]

    for i in range(m):
        for z in range(5):
            if ns[i] != 0:
                mus[i, z] /= ns[i]

    # Iterate moving boundaries
    changed = True
    iter = 0
    while changed and iter < maxIter:
        changed = False
        for xi in range(w):
            for y in range(h):
                x = xi
                s = S[y, x]

                x0 = x - 1 if x > 0 else x
                x1 = x + 1 if x < w - 1 else x
                y0 = y - 1 if y > 0 else y
                y1 = y + 1 if y < h - 1 else y

                T = [S[y, x0], S[y, x1], S[y0, x], S[y1, x]]
                if s == T[0] and s == T[1] and s == T[2] and s == T[3]:
                    continue

                # Compute error of each sp label, store best
                vs = np.array([float(x), float(y)] + [I[y, x, j] for j in range(3)], dtype=np.float32)
                es = np.array([E[y, x0], E[y, x1], E[y0, x], E[y1, x]], dtype=np.float32)

                dBest = 1e10
                sBest = T[0] + 1

                for i in range(4):
                    t = T[i]
                    if t == sBest:
                        continue

                    e = 1e10
                    for j in range(4):
                        if T[j] != t:
                            e = min(e, es[j])

                    ds = np.zeros(5, dtype=np.float32)
                    for j in range(5):
                        ds[j] = mus[t, j] - vs[j]
                        ds[j] *= ds[j]

                    d = (ds[0] + ds[1]) * sigx + (ds[2] + ds[3] + ds[4]) * sigc - e * sige
                    d += sigs / ns[t]

                    if d < dBest:
                        dBest = d
                        sBest = t

                # Assign pixel to closest sp, update new and old sp
                if sBest != s:
                    t = sBest
                    changed = True
                    ns[s] -= 1
                    ns[t] += 1
                    S[y, x] = t
                    for j in range(5):
                        mus[s, j] = (mus[s, j] * (ns[s] + 1) - vs[j]) / ns[s]
                    for j in range(5):
                        mus[t, j] = (mus[t, j] * (ns[t] - 1) + vs[j]) / ns[t]

        iter += 1

    return S


# Example usage
S = scipy.io.loadmat('Data_pre_sticky1/pre_sticky1_S.mat')
S = S['pre_sticky1_S']
I = scipy.io.loadmat('Data_pre_sticky1/pre_sticky1_I.mat')
I = I['pre_sticky1_I']
E = scipy.io.loadmat('Data_pre_sticky1/pre_sticky1_E0.mat')
E = E['pre_sticky1_E0']
prm = scipy.io.loadmat('Data_pre_sticky1/pre_sticky1_p.mat')
prm = prm['pre_sticky1_p'][0]
print("Original S:")
print(S)
h, w = S.shape

adjusted_S = sticky(S, h, w, I, E, prm)

# print("Adjusted S:")
# print(adjusted_S)
# Print size of S
print("Size of S:")
print(adjusted_S.shape)
# Print 10x10 region of S
# print("Printing the first 10 rows and columns of S")
# print(adjusted_S[:10, :10])

Original S:
[[    0   192   384 ... 48576 48768 48960]
 [    1   193   385 ... 48577 48769 48961]
 [    2   194   386 ... 48578 48770 48962]
 ...
 [  189   381   573 ... 48765 48957 49149]
 [  190   382   574 ... 48766 48958 49150]
 [  191   383   575 ... 48767 48959 49151]]
maxIter:  8
nThreads:  4
sigs:  0.0064
sigx:  0.0002
sige:  0.45
sigc:  0.05


/var/folders/f_/dg0mg7zs2yn54_n_kfn57g3w0000gn/T/ipykernel_72218/3339915618.py:97: RuntimeWarning: invalid value encountered in scalar divide
  mus[s, j] = (mus[s, j] * (ns[s] + 1) - vs[j]) / ns[s]
/var/folders/f_/dg0mg7zs2yn54_n_kfn57g3w0000gn/T/ipykernel_72218/3339915618.py:97: RuntimeWarning: divide by zero encountered in scalar divide
  mus[s, j] = (mus[s, j] * (ns[s] + 1) - vs[j]) / ns[s]


Size of S:
(192, 256)


In [7]:
# Load S_aftersitcky.mat using scipy.io.loadmat
file_path = 'Data_pre_sticky1/S_after_sticky_prelabel.mat'
S_after_sticky_prelabel = scipy.io.loadmat(file_path)
S_after_sticky_prelabel = S_after_sticky_prelabel['S_after_sticky_prelabel']
# Print indices where S_after_sticky_prelabel and adjusted S are different
print("Indices where S_after_sticky_prelabel and adjusted S are different")
print(np.where(S_after_sticky_prelabel != adjusted_S))
# Print the row, column and value where S_after_sticky_prelabel and adjusted S are different
indices = np.column_stack(np.where(S_after_sticky_prelabel != adjusted_S))
for index in indices:
    print(f"Row: {index[0]}, Column: {index[1]}, Value: {adjusted_S[index[0], index[1]]}, Original Value: {S_after_sticky_prelabel[index[0], index[1]]}")

Indices where S_after_sticky_prelabel and adjusted S are different
(array([62, 63, 63, 64, 64, 64, 65, 65, 66]), array([156, 156, 157, 156, 157, 158, 156, 157, 157]))
Row: 62, Column: 156, Value: 30209, Original Value: 30208
Row: 63, Column: 156, Value: 30209, Original Value: 30208
Row: 63, Column: 157, Value: 30209, Original Value: 30208
Row: 64, Column: 156, Value: 30209, Original Value: 30208
Row: 64, Column: 157, Value: 30209, Original Value: 30208
Row: 64, Column: 158, Value: 30209, Original Value: 30590
Row: 65, Column: 156, Value: 30209, Original Value: 30208
Row: 65, Column: 157, Value: 30209, Original Value: 30208
Row: 66, Column: 157, Value: 30209, Original Value: 30208


In [13]:
# Proof that relabel works
S_actual = scipy.io.loadmat('Data_pre_sticky1/S_calc.mat')
S_actual = S_actual['S_calc']
h, w = S_after_sticky_prelabel.shape
relabeled_S = relabel(S_after_sticky_prelabel, h, w)
print("Actual S print 10x10:")
print(S_actual)
print("Relabeled S print")
print(relabeled_S)
# Print indices where S_actual and relabeled S are different
print("Indices where S_actual and relabeled S are different")
# Print the row, column and value where S_actual and relabeled S are different
indices = np.column_stack(np.where(S_actual != relabeled_S))
for index in indices:
    print(f"Row: {index[0]}, Column: {index[1]}, Value: {relabeled_S[index[0], index[1]]}, Original Value: {S_actual[index[0], index[1]]}")
# Print how many labels are different
print("Number of labels that are different")
print(np.sum(S_actual != relabeled_S))

Actual S print 10x10:
[[   0    0    0 ... 2245 2287 2287]
 [   0    0    0 ... 2245 2287 2287]
 [   0    0    0 ... 2245 2287 2287]
 ...
 [  45   45   54 ... 2273 2273 2273]
 [  46   54   54 ... 2236 2236 2294]
 [  46   46   54 ... 2236 2236 2294]]
Relabeled S print
[[   0    0    0 ... 2245 2287 2287]
 [   0    0    0 ... 2245 2287 2287]
 [   0    0    0 ... 2245 2287 2287]
 ...
 [  45   45   54 ... 2273 2273 2273]
 [  46   54   54 ... 2236 2236 2294]
 [  46   46   54 ... 2236 2236 2294]]
Indices where S_actual and relabeled S are different
Number of labels that are different
0
